# Constructive Heuristic Template
# Flow Shop Scheduling Problem
Prof. María Angélica Salazar Aguilar

Selected Topics of Optimization

---
This notebook is a template for documenting a combinatorial optimization problem and its constructive heuristic. Follow the instructions in each section and provide the required content.



## 1. Team: TeamID

Provide the names, student IDs, and contact information of all team members below.

|Student ID      | Name  | Email             |
|----------------|------------|-------------------|
|2013939         | Aldo Sebastian Lopez Rivas | aldo.lopezrvs@uanl.edu.mx                  |
|2173850                |Josue Sebastian Cruz Cantu            |            josue.cruzcn@uanl.edu.mx       |
|2173891                |Iver Jair Salas Sanchez            |     iver.salass@uanl.edu.mx              |
|2014777                |Juan Carlos Sanchez Valencia|        juan.sanchezvln@uanl.edu.mx           |

## 2. General Description of the Problem

Flow-shop scheduling is an optimization problem in computer science and operations research. It is a variant of optimal job scheduling. In a general jobscheduling problem, we are given n jobs J1, J2, . . . , Jn of varying processing
times, which need to be scheduled on m machines with varying processing power,
while trying to minimize the makespan – the total length of the schedule (that
is, when all the jobs have finished processing).
In the specific variant known as flow-shop scheduling, each job contains
exactly m operations. The i-th operation of the job must be executed on the
i-th machine. No machine can perform more than one operation simultaneously.
For each operation of each job, execution time is specified.


## 3. Mathematical Formulation

**Flow Shop Scheduling Problem (FSSP)**

**Sets and data**  
- $J=\{1,\dots,n\}$ jobs, $M=\{1,\dots,m\}$ machines.  
- $p_{ij}\ge 0$: processing time of job $j$ on machine $i$.  
- A permutation (sequence) of jobs is denoted by $\pi=(\pi_1,\ldots,\pi_n)$.

**State (completion-time) recursions**  
Let $C_{i,\pi_k}$ be the completion time of job $\pi_k$ on machine $i$. Then

\begin{align}
C_{1,\pi_1} &= p_{1,\pi_1}, \\
C_{1,\pi_k} &= C_{1,\pi_{k-1}} + p_{1,\pi_k} \qquad (k=2,\ldots,n), \\
C_{i,\pi_1} &= C_{i-1,\pi_1} + p_{i,\pi_1} \qquad (i=2,\ldots,m), \\
C_{i,\pi_k} &= \max\{\, C_{i-1,\pi_k},\; C_{i,\pi_{k-1}} \,\} + p_{i,\pi_k} \qquad (i=2,\ldots,m;\; k=2,\ldots,n).
\end{align}

The **makespan** is $C_{\max} = C_{m,\pi_n}$.

**Objective**  
$$
\min_{\pi \in \mathcal{S}_n} \; C_{\max}(\pi) \;=\; C_{m,\pi_n}.
$$

where $\mathcal{S}_n$ is the set of all permutations of $n$ jobs (identical machine order for every job).

> *Note.* A full MILP can be written with binary assignment variables $x_{jk}\in\{0,1\}$ indicating job $j$ at position $k$ and time variables, but here we emphasize a constructive permutation heuristic.

---

### Pendulum Heuristic (used to construct $\pi$)

**Idea.** Select the first job based on the shortest processing time on machine 1, then sort the remaining jobs by total processing time $T_j=\sum_{i=1}^m p_{ij}$ (ascending) and place them alternately at the left and right ends of the sequence, starting from the right. This ensures the job with the smallest first-machine time begins the sequence, while the remaining jobs follow a pendulum pattern with lighter totals at the extremes and heavier totals toward the center.

**Steps**
1. Compute $T_j=\sum_{i=1}^m p_{ij}$ for all $j\in J$.
2. **Select the first job:** Choose job $j_1$ such that $p_{j_1,1} = \min_{j\in J} p_{j,1}$. In case of ties, select the job with the smallest $T_j$. Set $\pi_1 \leftarrow j_1$ and remove $j_1$ from $J$.
3. **Order remaining jobs:** Let $(j_2,\ldots,j_n)$ be the order of the remaining jobs such that $T_{j_2}\le \cdots \le T_{j_n}$. In case of ties in $T_j$, order by $p_{j,1}$ (ascending).
4. **Apply pendulum pattern:** Initialize two pointers $l\leftarrow 2$, $r\leftarrow n$. For $k=2$ to $n$:
   - if $k$ is even, set $\pi_r \leftarrow j_k$ and $r\leftarrow r-1$;
   - if $k$ is odd, set $\pi_l \leftarrow j_k$ and $l\leftarrow l+1$.

This yields a permutation $\pi$ where the first position holds the job with minimum first-machine time, and the remaining positions follow a pendulum layout with lighter jobs at the ends and heavier jobs near the center.


## 4. Pseudocode of the Proposed Constructive Heuristic

1. Start with the first job:
    - Look at all jobs and find the one that takes the least time on the first machine.
    - If there's a tie, pick the one with the shortest total time across all machines.
    - If still tied, pick the one that appears first in the list.
2. Sort the remaining jobs:
    - For all other jobs, sort them by their total time across all machines (shortest first).
    - If two jobs have the same total time, sort them by their time on the first machine.
    - If still tied, keep their original order.
3. Place the jobs like a pendulum:
    - Start with the first job you picked.
    - Take the next job from the sorted list and place it at the end of your sequence.
    - Take the next job and place it at the beginning.
    - Keep alternating: next job goes to the end, then beginning, and so on.
4. Final sequence:
    - The order you've created is your final schedule.


## 5. Description and Definition of Main Functions

Below are the core functions used in our solver. The **Pendulum heuristic** is the primary constructor; other heuristics are included only for comparison.

Our repository: https://github.com/Aselori/FlowShopConstructive 

---

### `solve_flow_shop(file_path: str) -> Optional[Dict[str, Any]]`  
*Module:* `main`  
- **Purpose:** Solves the Flow Shop Scheduling Problem using the Pendulum heuristic.  
- **Inputs:** Path to input file (.csv or Taillard .txt/.fsp).  
- **Outputs:** Dictionary containing solution details or None if an error occurred

---

### `read_instance(file_path: str) -> Tuple[List[List[float]], List[str]]`  
*Module:* `io_utils`  
- **Purpose:** Read a Flow Shop instance in CSV or Taillard TXT/FSP format.  
- **Inputs:** Path to input file (.csv or Taillard .txt/.fsp).  
- **Outputs:** Tuple of (processing-time matrix, job names).

---

### `read_csv_data(file_path: str) -> Tuple[List[List[float]], List[str]]`  
*Module:* `io_utils`  
- **Purpose:** Reads CSV files with automatic header detection. 

---

### `read_taillard_txt(file_path: str) -> Tuple[List[List[float]], List[str]]`  
*Module:* `io_utils`  
- **Purpose:** Parses Taillard-style TXT/FSP format files 

---

### `validate_processing_times(processing_times): List[List[float]]) -> None`  
*Module:* `io_utils`  
- **Purpose:** Validates the processing times matrix for logical consistency. 

---

### `print_data_summary(processing_times: List[List[float]], job_names: List[str]) -> None`  
*Module:* `io_utils`  
- **Purpose:** Displays a summary of the loaded instance. 

---

### `calculate_makespan(processing_times, sequence) -> float`  
*Module:* `makespan`  
- **Purpose:** Evaluate a sequence using the standard flow-shop completion-time recursion; returns the completion time of the last job on the last machine.  
- **Inputs:** Processing-time matrix, permutation.  
- **Outputs:** `makespan` (float).

---

### `print_sequence_analysis(processing_times, sequence, job_names=None) -> None`  
*Module:* `makespan`  
- **Purpose:** Convenience reporter: prints makespan, per-machine idle times, and utilization/efficiency for a given sequence.  
- **Inputs:** Matrix, permutation, optional job names.  
- **Outputs:** Console report (no return).

---

### `(utility for demos) create_sample_data(file_path, num_jobs=5, num_machines=3) -> None`  
*Module:* `main`  
- **Purpose:** Generate a small synthetic instance for testing and demos.  
- **Inputs:** Output path, numbers of jobs/machines.  
- **Outputs:** CSV file written to disk.

---

> **Note:** Additional constructive heuristics (NEH, SPT/LPT, Palmer, CDS/Johnson) are implemented to enable fair comparisons but are not the primary focus of this study.


In [6]:
# 6. Implementation of the Constructive Heuristic

def pendulum_heuristic(processing_times: List[List[float]]) -> List[int]:
    if not processing_times:
        return []
    
    num_jobs = len(processing_times)
    num_machines = len(processing_times[0]) if num_jobs > 0 else 0

    # Precompute totals and machine-1 times for tie-breaking
    totals = [sum(times) for times in processing_times]
    m1_times = [times[0] for times in processing_times] if num_machines > 0 else [0] * num_jobs

    # First job selection with tie-breaking
    first_job = min(range(num_jobs), key=lambda j: (m1_times[j], totals[j], j))

    sequence = [-1] * num_jobs
    sequence[0] = first_job

    # Sort remaining jobs with tie-breaking
    remaining = [j for j in range(num_jobs) if j != first_job]
    remaining.sort(key=lambda j: (totals[j], m1_times[j], j))

    # Pendulum placement
    left = 1
    right = num_jobs - 1
    for k, job_idx in enumerate(remaining):
        if k % 2 == 0:
            sequence[right] = job_idx
            right -= 1
        else:
            sequence[left] = job_idx
            left += 1

    return sequence



In [7]:
## 7. Main Function to Run the Code

## ====================
# MAIN FUNCTION TO RUN THE CODE
# ====================
def main():
    """
    Example usage of the pendulum heuristic in a Jupyter notebook.
    This cell should be able to run independently after the heuristic cell.
    """
    # Example processing times (replace with your data)
    # Format: Each row is a job, each column is a machine
    processing_times = [
        [5, 2, 3],  # Job 0
        [2, 4, 6],  # Job 1
        [4, 3, 2],  # Job 2
        [3, 2, 4]   # Job 3
    ]
    job_names = [f"Job_{i+1}" for i in range(len(processing_times))]
    
    # Run the heuristic
    print("Running Pendulum Heuristic...")
    sequence = pendulum_heuristic(processing_times)
    
    # Display results
    print("\n=== RESULTS ===")
    print(f"Optimal sequence: {[job_names[i] for i in sequence]}")
    print(f"Sequence indices: {sequence}")
    
    # Calculate makespan
    num_jobs = len(sequence)
    num_machines = len(processing_times[0]) if num_jobs > 0 else 0
    completion = [[0] * (num_machines + 1) for _ in range(num_jobs + 1)]
    
    for i in range(1, num_jobs + 1):
        job_idx = sequence[i-1]
        for j in range(1, num_machines + 1):
            completion[i][j] = max(completion[i-1][j], completion[i][j-1]) + \
                              processing_times[job_idx][j-1]
    
    makespan = completion[num_jobs][num_machines]
    print(f"Makespan: {makespan}")

# Run the main function
if __name__ == "__main__":
    main()

Running Pendulum Heuristic...

=== RESULTS ===
Optimal sequence: ['Job_2', 'Job_3', 'Job_1', 'Job_4']
Sequence indices: [1, 2, 0, 3]
Makespan: 21


## 8. Computational Results and Discussion
(https://github.com/chneau/go-taillard/tree/master/pfsp/instances)


| Instance | Objective Value Heuristic | Objective Value (Optimal) | Gap (%) |
|----------|----------|----------------|----------|
|     1     |    1911      |      1582          |     20.79%     |
|     2     |     2020     |        1659        |       21.76%   |
|     3     |      2010    |          1496      |     34.41%     |
|     4     |      2570    |         2297       |     11.88%     |
|     5     |      2491    |         2100       |     18.62%     |
|     6     |      2637    |        2326        |     13.37%     |
|     7     |      4019    |        3025        |      32.87%    |
|     8     |     3541     |         2892       |     22.43%     |
|     9     |      3767    |        2864        |     31.52%     |
|    10     |      6706    |        5770        |      16.22%    |
|    11     |      6884    |        5349        |     28.69%     |
|    12     |      6574    |       5677         |    15.80%      |
|    13     |      7946    |        6286        |     26.41%     |
|    14     |      7875    |      6241          |    26.18%      |
|    15     |      7763    |        6329        |     22.65%     |

**Discussion:**
- Regarding the effectiveness of the heuristic, the results show an average gap of about 23%, with some instances reaching as low as 11.88%. This indicates a moderately consistent performance, though the method still leaves a notable margin from the optimal.

- When comparing across instances, no clear trend of deterioration with problem size is observed, which suggests good scalability. Outliers such as instances 3, 7 and 9, however, highlight situations where the heuristic struggles, and including computation times would help balance quality against efficiency.

- In terms of patterns, results are widely distributed, with gaps ranging from 11.88% to 34.41%, with only a few extreme cases. This points to a stable baseline that could be improved by adding local refinement or multi-start strategies to reduce the worst gaps and bring the average closer to competitive levels.


## 9. General Conclusions

The heuristic developed in this work was intentionally kept simple and implemented relatively quickly during class. While it provides feasible solutions with an average gap of around 23%, its performance was slightly worse than initially expected. Part of this can be attributed to missing logic that was overlooked during implementation, which limited the quality of the results.

A key lesson learned is that even a basic heuristic can provide consistent approximations, but careful design choices and refinement are crucial to avoid systematic weaknesses. The analysis also highlighted the importance of testing across diverse instances, since a few outliers revealed specific scenarios where the method struggles.

For future work, there are several straightforward improvements that could be explored, such as adding local search steps, introducing randomized restarts, or refining the construction logic. These additions are expected to reduce the gap significantly and would serve as natural next steps if the goal were to turn this initial prototype into a more competitive approach.

## 10. Revised References

- Chneau, C. (n.d.). go-taillard: PFSP instances. GitHub. Retrieved October 1, 2025, from https://github.com/chneau/go-taillard/tree/master/pfsp/instances

